In [1]:
import os

os.environ["COHERE_API_KEY"] = 'fOKNiUz8cq3c7LfrrzrX7E70vgGpZStUEgdt3Ohj'

In [65]:
from langchain_cohere import ChatCohere
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [3]:
db_user = "root"
db_password = "Donsurya23!"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.get_usable_table_names())
# db.run("SELECT * FROM discounts")

['discounts', 't_shirts']


In [4]:
llm = ChatCohere(model="command-r-plus")

In [5]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many discounts are there"})
response

'```sql\nSELECT COUNT(*) AS `num_discounts` FROM `discounts`;\n```'

In [6]:
clean_response = response[7:].split(";")[0]
clean_response
db.run(clean_response)

'[(9,)]'

In [7]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | (lambda x: (print(x), x[7:].split(";")[0])[1]) | execute_query

In [8]:
qns1 = chain.invoke({"question": "How many tshirts do we have left for nike in extra small size and white color?"})
qns1

```sql
SELECT `stock_quantity` 
FROM `t_shirts` 
WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS';
```


'[(20,)]'

In [9]:
# qns2 = chain.invoke({"question": "How much is the price of the inventory for all small size tshirts?"})
qns2 = chain.invoke({"question": "SELECT SUM(price*stock_quantity) FROM atliq_tshirts.t_shirts WHERE size = 'S';"})
qns2

```sql
SELECT SUM(`price` * `stock_quantity`) AS `total_value`
FROM `t_shirts`
WHERE `size` = 'S';
```


"[(Decimal('24886'),)]"

In [10]:
# qns3 = chain.invoke({"question": "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"})
qns3 = chain.invoke({"question": """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """})
qns3

```sql
SELECT SUM(a.total_amount * ((100 - COALESCE(d.pct_discount, 0)) / 100)) AS total_revenue
FROM (
    SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id
    FROM t_shirts
    WHERE brand = 'Levi'
    GROUP BY t_shirt_id
) a
LEFT JOIN discounts d ON a.t_shirt_id = d.t_shirt_id;
```


"[(Decimal('22286.300000'),)]"

In [31]:
# qns4 = chain.invoke({"question": "select SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi';"})
qns4 = db.run("select SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi';")
qns4

"[(Decimal('23146'),)]"

In [33]:
# qns5 = chain.invoke({"question": "How many white color Levi's t shirts we have available?"})
# qns5 = chain.invoke({"question": "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"})
qns5 = db.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")
qns5

"[(Decimal('170'),)]"

In [36]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [45]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

load INSTRUCTOR_Transformer
max_seq_length  512


/home/donc/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [47]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query [(20,)]",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query [(Decimal('24886'),)]",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query [(Decimal('22286.300000'),)]",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_qua

In [53]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [59]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '[(20,)]',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': "[(Decimal('170'),)]",
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [63]:
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [67]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [77]:
new_write_query = create_sql_query_chain(llm, db, prompt=few_shot_prompt)

In [81]:
new_write_query.invoke({"question": "how many tshirts are there?"})

'SQLQuery: SELECT count(DISTINCT `t_shirt_id`) AS total_tshirts FROM t_shirts;\nSQLResult:'

In [83]:
new_chain = new_write_query | (lambda x: (print(x), x[10:].split(";")[0])[1]) | execute_query

In [87]:
new_chain.invoke({"question": "How many white color Levi's shirt I have?"})

SQLQuery: SELECT sum(`stock_quantity`) AS total_white_levi_shirts FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White';
SQLResult:


"[(Decimal('170'),)]"